# import def func

In [1]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [57]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20):
    col_src = [
        "name", "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

from JSONData import tdx_hdf5_api as h5a
def get_tdx_all_from_h5(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d':
        dl='60'
    else:
        dl='200'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    return h5


def get_tdx_all_from_h5(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
    if end is not None:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
            '_' + end.replace('-', '') + '_' + 'all'
    else:
        h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    return h5

## init data

### get_tdx_all

In [3]:
h5_day=get_tdx_all_from_h5(resample='d')

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


### sina

In [4]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-17',
 code
 002560    通达股份
 Name: name, dtype: object)

In [5]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/18/25 16:30:44'

code
603757   2025-06-18 15:00:01
002538   2025-06-18 15:00:00
Name: ticktime, dtype: datetime64[ns]

90.74918293158213

#### 实时成交

In [6]:
def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if root is None and store is not None:
        root = store.keys()[0].replace("/", "")
    df = store[root]
    df = apply_col_toint(df)
    store.close()
    return df

In [7]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
h5.shape
h5code='603019'
h5.loc[h5code][-3:]
code_muti = '603019'
# h5.loc[code_muti][:2]

freq = 'D'
# freq = '5M'
startime = '09:25:00'
endtime = '15:01:00'

time_ratio = cct.get_work_time_ratio()
time_ratio
run_col = ['close', 'volume']
# run_col = ['close']
mdf = cct.get_limit_multiIndex_freq(
    h5, freq=freq.upper(),
    col=run_col, start=startime, end=endtime, code=None)
allvol = mdf.volume.sum()
allvol = round(allvol / 10 / 1000 / 1000,2)
print("总量:%s 亿"%(allvol))
mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
#全局占比
mdf['volume'] = mdf.amount.apply(
    lambda x: round(x / allvol * 100 / time_ratio, 2))
mdf=mdf.reset_index().set_index('code')
mdf.loc[h5code]
str(h5.index.get_level_values(1)[-1])[:10]

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(2699146, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-18 14:59:48,67.54,67.97,66.88,67.91,20060073,67.62
2025-06-18 15:00:03,67.40,67.97,66.88,67.91,20369873,67.72
2025-06-18 15:00:03,67.40,67.97,66.88,67.91,20369873,67.40


1.0

总量:9756.35 亿


ticktime    2025-06-19 00:00:00
close                 67.530481
volume                     0.14
vol                       20.37
amount                    13.76
Name: 603019, dtype: object

'2025-06-18'

#### 读取实时成交个股

In [9]:
nowdf = mdf.sort_values(by=['volume'], ascending=[0])
# nowdf.shape, df.shape, code_idx.shape
# top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
top20_nd = get_sina_data_cname(top20_nd)
nowdf.shape, top20_nd

((5418, 5),
         volume  name
 code                
 302132    0.41   NaN
 300308    0.84  中际旭创
 300773    0.66   拉卡拉
 300502    0.63   新易盛
 300468    0.63  四方精创
 300476    0.59  胜宏科技
 300180    0.54  华峰超纤
 002366    0.53  融发核电
 002463    0.51  沪电股份
 002384    0.46  东山精密
 002104    0.46  恒宝股份
 600519    0.45  贵州茅台
 300059    0.44  东方财富
 002475    0.40  立讯精密
 002261    0.38  拓维信息
 002594    0.37   比亚迪
 000617    0.35  中油资本
 300255    0.35  常山药业
 300251    0.34  光线传媒
 600418    0.34  江淮汽车)

##### 合并实时成交量及SinaDM

In [10]:
sina_dm.shape

(5180, 41)

In [11]:
# mdf[:1]
sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)

## resample 多周期

### resample 3d

In [12]:
h5_3d = get_tdx_all_from_h5(resample='3d')
tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [21]:
tdx_3d.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
show_tdx_data(tdx_select)

(5182, 218)

Count:1384 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000001,平安银行,2025-04-09,11.77,11.34,10.12,1.3,11.74,10.12,0.10,1.3
000021,深科技,2025-04-09,17.33,18.88,14.74,0.1,17.19,14.74,-0.10,0.0
000034,神州数码,2025-04-09,36.44,42.44,31.34,0.0,36.28,31.34,-0.10,0.1
000045,深纺织Ａ,2025-04-09,11.09,11.51,8.33,0.0,10.92,8.33,-0.90,1.2
000049,德赛电池,2025-04-09,21.67,23.00,17.91,0.0,21.47,17.91,0.10,0.0
000050,深天马Ａ,2025-04-09,8.04,8.39,6.70,0.0,7.99,6.70,-0.10,0.0
000063,中兴通讯,2025-04-09,31.52,34.12,28.05,0.1,31.42,28.05,0.00,0.1
000066,中国长城,2025-03-31,14.45,16.48,13.01,0.0,14.46,13.68,0.00,0.0
000100,TCL科技,2025-04-09,4.37,4.45,3.68,0.1,4.36,3.68,0.00,0.0


In [22]:
t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')

In [23]:
show_tdx_data(t3d_code)

Count:407 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000001,平安银行,2025-04-09,11.77,11.34,10.12,1.30,11.74,10.12,0.10,1.3
000159,国际实业,2025-04-09,5.88,5.68,4.21,39.19,5.91,4.21,-0.90,0.0
000338,潍柴动力,2025-05-27,15.50,16.41,14.08,1.20,15.46,14.67,1.30,0.1
000407,胜利股份,2025-04-09,3.56,3.37,2.76,30.59,3.52,2.76,0.00,0.1
000408,藏格矿业,2025-04-09,40.91,37.66,31.00,1.20,40.84,31.00,7.30,0.0
000430,ST张家界,2025-04-25,8.37,9.17,6.60,0.00,8.20,6.67,0.00,1.2
000518,*ST四环,2025-05-06,2.70,2.66,1.72,1.20,2.70,1.81,33.41,12.9
000519,中兵红箭,2025-04-18,17.55,18.77,15.17,1.20,17.78,16.30,0.00,0.2
000520,凤凰航运,2025-04-09,5.54,5.50,3.44,31.20,5.59,3.44,0.10,0.0


### resample W

In [28]:
h5_w = get_tdx_all_from_h5(resample='w')
tdx_w = cct.combine_dataFrame(h5_w,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [46]:
tdx_w.shape
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
tdx_select_w = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')[:10]
show_tdx_data(tdx_select_w)

(5180, 219)

Count:10 219


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000029,深深房Ａ,2025-01-10,19.8,18.0,12.8,1.2,20.0,13.9,23.8,1.3
000513,丽珠集团,2025-03-07,36.0,38.2,32.9,1.2,36.4,34.1,0.1,1.2
000533,顺钠股份,2024-11-22,7.2,7.4,3.9,1.2,7.2,3.9,1.2,1.2
000599,青岛双星,2025-01-10,5.0,5.3,4.1,1.2,5.0,4.1,0.1,1.3
000701,厦门信达,2025-01-03,7.7,5.7,3.7,50.6,7.7,4.3,10.1,0.0
000766,通化金马,2024-11-22,27.1,20.0,13.6,46.5,27.2,13.7,1.3,20.5
000831,中国稀土,2025-01-03,36.9,34.2,26.9,1.2,36.8,26.9,1.1,1.3
000919,金陵药业,2025-01-10,6.9,7.1,5.9,1.2,6.9,6.3,0.0,0.1
000963,华东医药,2025-01-10,42.7,42.8,30.4,1.2,43.4,30.4,28.4,8.9


In [25]:
tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
show_tdx_data(tw_code)

Count:535 219


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000001,平安银行,2025-04-11,11.77,11.61,10.12,1.30,11.74,10.12,1.3,1.20
000159,国际实业,2025-04-11,5.88,7.67,4.21,26.09,5.91,4.21,4.1,-0.10
000338,潍柴动力,2024-12-20,15.50,16.59,12.97,1.20,15.46,13.07,0.1,0.00
000407,胜利股份,2025-04-11,3.56,3.72,2.76,26.50,3.52,2.76,0.1,-0.10
000408,藏格矿业,2024-12-20,40.91,36.81,26.75,1.20,40.84,26.75,0.0,0.00
000428,华天酒店,2025-01-10,3.30,3.94,2.73,0.00,3.28,2.90,1.3,-0.10
000430,ST张家界,2025-01-03,8.37,9.17,5.56,1.20,8.20,5.80,1.2,0.10
000504,*ST生物,2025-04-30,8.74,10.89,5.43,15.30,8.80,5.43,1.3,1.20
000518,*ST四环,2025-04-30,2.70,3.71,1.72,1.20,2.70,1.91,1.3,1.20


### resample M

In [30]:
h5_m = get_tdx_all_from_h5(resample='m')
tdx_m = cct.combine_dataFrame(h5_m,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [31]:
tm_code = tdx_m.query('lasth1d > lasth2d > lasth3d and close >= high4')
show_tdx_data(tm_code)

Count:766 218


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
000001,平安银行,2025-04-30,11.77,11.60,9.00,1.30,11.74,10.12,-1.1,1.20
000010,美丽生态,2024-09-30,3.05,3.03,1.38,30.00,3.03,1.64,-1.1,1.10
000017,深中华A,2025-02-28,6.80,7.12,5.06,1.20,6.80,5.50,0.0,0.10
000020,深华发Ａ,2025-04-30,15.16,15.72,9.00,1.30,14.57,9.22,-1.1,-0.90
000159,国际实业,2025-04-30,5.88,7.44,4.83,1.30,5.91,4.21,0.0,1.10
000407,胜利股份,2024-09-30,3.56,3.45,2.47,1.30,3.52,2.60,0.0,-0.90
000408,藏格矿业,2024-12-31,40.91,35.84,21.14,35.81,40.84,26.75,0.0,1.20
000426,兴业银锡,2024-12-31,16.30,14.21,9.50,45.09,16.27,11.10,0.0,0.20
000513,丽珠集团,2025-02-28,35.62,39.56,32.97,1.20,35.53,32.97,-1.0,-1.10


### resample D

In [32]:
h5_d = get_tdx_all_from_h5(resample='d')
tdx_d = cct.combine_dataFrame(h5_d,sinadm)

keys:['/low_3d_60_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [58]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
td_code = tdx_d.query('ma51d > ma10d and ral > 3 and low >ma10d')
show_tdx_data(td_code,limit=5)

Count:763 220


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
605123,派克新材,2025-04-07,68.88,67.81,53.88,27.3,68.69,50.38,-0.1,0.1
688668,鼎通科技,2025-04-07,65.69,60.69,46.59,0.1,64.38,34.91,1.2,1.2
600617,国新能源,2025-04-07,2.91,2.69,2.52,36.0,2.99,2.25,1.3,0.1
600379,宝光股份,2025-04-08,11.13,11.89,9.33,0.0,11.02,8.29,0.1,0.1
603706,东方环宇,2025-04-09,22.31,19.41,18.47,-1.0,21.45,15.60,17.3,1.2


In [59]:
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))

In [60]:
show_tdx_data(td_code.loc[code_f])

Count:138 220


,name,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,
601107,四川成渝,2025-03-28,6.37,6.25,5.33,1.30,6.35,4.70,1.20,0.1
920088,科力股份,2025-04-07,55.03,31.66,25.09,54.19,58.56,19.12,35.31,1.2
002109,兴化股份,2025-04-08,4.22,3.50,3.24,-1.00,4.20,2.79,15.20,0.1
301528,多浦乐,2025-04-07,55.25,52.78,43.88,1.20,56.03,36.22,1.30,0.1
002130,沃尔核材,2025-04-08,23.66,22.81,18.58,0.10,23.11,16.05,0.10,0.0
300348,长亮科技,2025-04-07,15.50,14.43,12.12,1.30,15.70,10.32,0.00,1.2
002235,安妮股份,2025-04-08,8.06,6.27,5.15,0.00,7.64,4.33,17.30,0.1
300635,中达安,2025-04-07,11.25,11.09,9.73,0.10,11.26,8.01,1.20,0.1
688013,天臣医疗,2025-04-07,27.50,26.88,19.50,1.20,27.36,16.19,1.20,0.1
